In [46]:
using ARCHModels
using DataFrames
using LinearAlgebra
using HTTP
using Gumbo
using Pickle
using Cascadia
using YFinance

In [57]:
url = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
resp = HTTP.get(url)
html = String(resp.body)
parsed_html = parsehtml(html)
root = parsed_html.root
table_select = Selector("#constituents")
symbol_select = Selector(".external.text")
tables  = eachmatch(sm, root)
securities = eachmatch(symbol_select, tables[1])
tickers = map(String, text.(securities))
tickers = map(s -> replace(s, '.' => '-'), tickers)

503-element Vector{String}:
 "MMM"
 "AOS"
 "ABT"
 "ABBV"
 "ACN"
 "ADBE"
 "AMD"
 "AES"
 "AFL"
 "A"
 "APD"
 "ABNB"
 "AKAM"
 ⋮
 "WST"
 "WDC"
 "WY"
 "WMB"
 "WTW"
 "WYNN"
 "XEL"
 "XYL"
 "YUM"
 "ZBRA"
 "ZBH"
 "ZTS"

In [58]:
prices_stack = get_prices.(tickers,startdt="2000-01-01", enddt="2024-01-01")
df = vcat([DataFrame(i) for i in prices]...)
prices = unstack(df, :timestamp, :ticker, :adjclose)

┌ Error: Exception while generating log record in module YFinance at C:\Users\guibi\.julia\packages\YFinance\J0e1z\src\Prices.jl:171
│   exception =
│    KeyError: key :finance not found
│    Stacktrace:
│      [1] getindex
│        @ .\dict.jl:498 [inlined]
│      [2] get(obj::JSON3.Object{Vector{UInt8}, Vector{UInt64}}, key::Symbol)
│        @ JSON3 C:\Users\guibi\.julia\packages\JSON3\jSAdy\src\JSON3.jl:87
│      [3] getproperty(obj::JSON3.Object{Vector{UInt8}, Vector{UInt64}}, prop::Symbol)
│        @ JSON3 C:\Users\guibi\.julia\packages\JSON3\jSAdy\src\JSON3.jl:127
│      [4] macro expansion
│        @ .\logging.jl:372 [inlined]
│      [5] get_prices(symbol::String; range::String, interval::String, startdt::String, enddt::String, prepost::Bool, autoadjust::Bool, timeout::Int64, throw_error::Bool, exchange_local_time::Bool)
│        @ YFinance C:\Users\guibi\.julia\packages\YFinance\J0e1z\src\Prices.jl:171
│      [6] get_prices
│        @ C:\Users\guibi\.julia\packages\YFinance\J0e

LoadError: MethodError: no method matching length(::DataFrame)

[0mClosest candidates are:
[0m  length([91m::Combinatorics.FixedPartitions[39m)
[0m[90m   @[39m [32mCombinatorics[39m [90mC:\Users\guibi\.julia\packages\Combinatorics\Udg6X\src\[39m[90m[4mpartitions.jl:96[24m[39m
[0m  length([91m::DataStructures.EnumerateAll[39m)
[0m[90m   @[39m [33mDataStructures[39m [90mC:\Users\guibi\.julia\packages\DataStructures\95DJa\src\[39m[90m[4mmulti_dict.jl:96[24m[39m
[0m  length([91m::Cmd[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mprocess.jl:678[24m[39m
[0m  ...


In [59]:
prices

Row,timestamp,MMM,AOS,ABT,ABBV,ACN
,DateTime,Float64?,Float64?,Float64?,Float64?,Float64?
1,2000-01-03T14:30:00,19.9207,2.32224,8.32724,missing,missing
2,2000-01-04T14:30:00,19.1291,2.28878,8.08932,missing,missing
3,2000-01-05T14:30:00,19.6832,2.28208,8.07445,missing,missing
4,2000-01-06T14:30:00,21.2663,2.24862,8.35698,missing,missing
5,2000-01-07T14:30:00,21.6884,2.34232,8.4462,missing,missing
6,2000-01-10T14:30:00,21.5829,2.36908,8.38672,missing,missing
7,2000-01-11T14:30:00,21.2135,2.34232,8.26776,missing,missing
8,2000-01-12T14:30:00,21.2663,2.26201,8.14403,missing,missing
9,2000-01-13T14:30:00,21.2663,2.29547,8.06931,missing,missing
